In [1]:
import numpy as np
from sklearn import metrics
from collections import defaultdict
import re

In [2]:
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [3]:
from sklearn.preprocessing import StandardScaler


In [4]:
from sklearn.cluster import DBSCAN,KMeans

Данные предобработаны так:

    удаляются все цифры и знаки препинания, так как много номеров документов и телефонов, что сниает информативность
    текста    
    Далее строим все 1-грамы и смотрим на частотность слов
    Так же смотрим ни 2-грамы

In [5]:
ds = pd.read_csv("hillary-clinton-emails/Emails.csv")
ds = ds.fillna('')
persons = pd.read_csv("hillary-clinton-emails/Persons.csv")
ds.ExtractedBodyText = ds.ExtractedBodyText.apply(lambda x: re.sub(r"\d"," ", x))
#ds.head()

In [6]:
vectorizer = CountVectorizer(ngram_range=(1,1),stop_words='english')
X = vectorizer.fit_transform(ds.ExtractedBodyText)
features = vectorizer.get_feature_names()
counts = np.sum(X, axis=0)
con = []
for i in range(counts.shape[1]):
    con.append(counts[0, i])
a= zip(con, features)
a.sort(reverse=True)
a[:10]

[(2657, u'state'),
 (2393, u'pm'),
 (1359, u'secretary'),
 (1258, u'obama'),
 (1241, u'said'),
 (1129, u'president'),
 (1125, u'office'),
 (1106, u'new'),
 (1027, u'time'),
 (972, u'house')]

In [7]:
vectorizer = CountVectorizer(ngram_range=(2,2),stop_words='english', min_df=5)
X = vectorizer.fit_transform(ds.ExtractedBodyText)
features = vectorizer.get_feature_names()
counts = np.sum(X, axis=0)
con = []
for i in range(counts.shape[1]):
    con.append(counts[0, i])
a= zip(con, features)
a.sort(reverse=True)
a[:10]

[(761, u'state gov'),
 (460, u'secretary office'),
 (455, u'state department'),
 (449, u'united states'),
 (424, u'white house'),
 (315, u'en route'),
 (299, u'pm secretary'),
 (293, u'department state'),
 (284, u'office pm'),
 (248, u'secretary state')]

Подготовим набор данных для обработки nltk. Нужно получить один массив, со словами-элементами:

In [8]:
ds_for_bigram = ds.ExtractedBodyText.apply(lambda x: re.sub("[^a-zA-Z]", " ", x.lower()).split())
plain_ds = []
for item in ds_for_bigram:
    plain_ds.extend(item)

In [9]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(plain_ds)
finder.apply_freq_filter(5)
finder.nbest(bigram_measures.pmi, 10)

[('chico', 'menashe'),
 ('golestan', 'gilan'),
 ('ilam', 'bushehr'),
 ('khuzestan', 'zanjan'),
 ('nagorno', 'karabakh'),
 ('semnan', 'sistan'),
 ('zanjan', 'semnan'),
 ('buenos', 'aires'),
 ('elliott', 'trudeau'),
 ('fars', 'qazvin')]

In [10]:
vectorizer = TfidfVectorizer(ngram_range=(1,1),stop_words='english', min_df=4)
X = vectorizer.fit_transform(ds.ExtractedBodyText)

In [11]:
X.shape

(7945, 8059)

In [12]:
model = KMeans(n_clusters=7, random_state=42)
svd = TruncatedSVD(n_components=500, random_state=123)
features = svd.fit_transform(X)
preds = model.fit_predict(features)
print(preds[:30])


[1 1 1 1 3 1 1 3 2 1 2 1 2 1 1 1 1 1 1 3 3 3 5 1 1 1 1 1 1 1]


In [13]:
ind = 0

In [14]:
#print preds[214]
for i in range(7945):
    if preds[i]==4:
        if i > ind:
            ind = i
            break
print ind
print ds.ExtractedBodyText[ind]

487
[mailto
RELEASE IN PART
B 


In [15]:
right = []
fyi = []
rel = []
fifth = []
six = []
first = []
for i in range(7945):
    if preds[i]==0:
        right.append(i)
    if preds[i] == 1:
        first.append(i)
    if preds[i] == 2:
        fyi.append(i)
    if preds[i] == 4:
        rel.append(i)
    if preds[i] == 5:
        fifth.append(i)
    if preds[i] == 6:
        six.append(i)
print map(len, [right, fyi, rel, fifth, six, first])

[38, 450, 53, 108, 175, 6503]


В результате кластеризации выделились отдельные классы коротких сообщений, вроде:
-"Ok"
-"pls print"
-"fyi"
-"you right"
-"release in part b (press release)"
etc.
Так получается, потому что таких писем достаточно много датасете. 
На мой взгляд результат кластеризации является весьма интерпретируемым.
Что бы точнее произвести кластеризацию, необходимо исключить из датасета короткие сообщения, и провести повторную кластеризацию.
